In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
data = pd.read_csv("2023-24 Advanced Stats.xlsx - Sheet1.csv")

In [15]:
data = data[data['MP'] > 1300]

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV


# Select defensive-related columns
defensive_cols = ["STL%", "DRB%", "BLK%", "DWS", "DBPM"]
df_def = data[defensive_cols].dropna()

# Create a 'Total' column by summing across the selected defensive features
df_def['Total'] = df_def.sum(axis=1)

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_def)

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Prepare target variable — we use 'DBPM' as the defensive performance proxy
y = df_def['Total'].values

# Use LassoCV to select important principal components
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_pca, y)

# Get the Lasso-selected components and their weights
lasso_coef = lasso.coef_
selected_pcs = np.where(lasso_coef != 0)[0]

# Compute the weighted PC score
weighted_score = X_pca[:, selected_pcs] @ lasso_coef[selected_pcs]

# Normalize the score to 0–100
min_score, max_score = weighted_score.min(), weighted_score.max()
normalized_score = 100 * (weighted_score - min_score) / (max_score - min_score)

# Add the final score to the dataframe
df_result = data.loc[df_def.index].copy()
df_result['Defensive Impact Score'] = normalized_score

top_5_by_pos = df_result.sort_values('Defensive Impact Score', ascending = False).groupby('Pos').head(5)
# Show only relevant columns
top_5_by_pos_display = top_5_by_pos[['Player', 'Team', 'Pos', 'Defensive Impact Score']].sort_values(by = 'Defensive Impact Score', ascending=False).reset_index(drop=True)

#export_df = df_result[["Player", "Defensive Impact Score"]].sort_values(ascending=False, by = "Defensive Impact Score")
export_df = top_5_by_pos_display

/opt/miniconda3/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/opt/miniconda3/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/opt/miniconda3/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/var/folders/m4/9g845_mn0m97yp6bjm5ld8x80000gn/T/ipykernel_60852/1953840357.py:33: RuntimeWarning: divide by zero encountered in matmul
  weighted_score = X_pca[:, selected_pcs] @ lasso_coef[selected_pcs]
/var/folders/m4/9g845_mn0m97yp6bjm5ld8x80000gn/T/ipykernel_60852/1953840357.py:33: RuntimeWarning: overflow encountered in matmul
  weighted_score = X_pca[:, selected_pcs] @ lasso_coef[selected_pcs]
/var/folders/m4/9g845_mn0m97yp6bjm5ld8x80000gn/T/ipykernel_60852/1953840357.py

In [30]:
export_df.to_csv("/Users/siddharth/Desktop/Lasso_PCA.csv", index = False)